# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [2]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [3]:
# Drop unnecessary columns
new_df = df.drop(columns=['timestamp'])

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [4]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
from surprise import Reader, Dataset
# read in values as Surprise dataset
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(new_df, reader)
data


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [5]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [6]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [7]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid = {
    'n_factors': [20, 100],
    'n_epochs': [5, 10],
    'lr_all': [0.002, 0.005],
    'reg_all': [0.05, 0.6]
}
gs = GridSearchCV(SVD, param_grid=param_grid, n_jobs=-1, joblib_verbose=5)
gs.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    2.4s
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   15.5s finished


In [8]:
# print out optimal parameters for SVD after GridSearch
print('Best RMSE:', gs.best_score['rmse' ])

# Best parameters
print('Best params:', gs.best_params['rmse' ])

Best RMSE: 0.8778022067864594
Best params: {'n_factors': 20, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.05}


In [9]:
# cross validating with KNNBasic
sim_pearson = {"name": "pearson"}

# Initialize the KNNBasic algorithm with the specified similarity options
basic_pearson = KNNBasic(sim_options=sim_pearson)

# Cross-validate with 5 folds, using RMSE as the metric
results = cross_validate(basic_pearson, data, measures=['rmse'], cv=5, verbose=True)
print(results)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9741  0.9759  0.9748  0.9727  0.9666  0.9728  0.0033  
Fit time          0.43    0.44    0.45    0.48    0.47    0.45    0.02    
Test time         1.55    1.87    1.53    2.29    1.67    1.78    0.28    
{'test_rmse': array([0.97406541, 0.97590612, 0.97479464, 0.97271347, 0.96662503]), 'fit_time': (0.42798924446105957, 0.44288063049316406, 0.4472522735595703, 0.4770019054412842, 0.46901392936706543), 'test_time': (1.5490391254425049, 1.8721392154693604, 1.53463

In [10]:
# print out the average RMSE score for the test set
average_rmse = results['test_rmse'].mean()
print(f"Average RMSE score for the test set: {average_rmse:.4f}")

Average RMSE score for the test set: 0.9728


In [11]:
# cross validating with KNNBaseline
sim_pearson = {"name": "pearson"}

# Initialize the KNNBaseline algorithm with the specified similarity options
knn_baseline = KNNBaseline(sim_options=sim_pearson)

# Cross-validate with 5 folds, using RMSE as the metric
results1 = cross_validate(knn_baseline, data, measures=['rmse'], cv=5, verbose=True)

Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8799  0.8711  0.8810  0.8877  0.8695  0.8778  0.0067  
Fit time          0.84    0.84    0.81    0.97    0.83    0.86    0.06    
Test time         2.03    2.07    2.09    2.71    2.02    2.18    0.26    


In [12]:
# print out the average score for the test set
average_rmse = results1['test_rmse'].mean()
print(f"Average RMSE score for the test set: {average_rmse:.4f}")

Average RMSE score for the test set: 0.8778


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [13]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [14]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [15]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [16]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=3.0224848284553385, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [17]:
def movie_rater(movie_df,num, genre=None):
    rating_list = []
    user_id = 1000
    
    if genre:
        genre_movies = movie_df[movie_df['genres'].str.contains(genre)]
    else:
        genre_movies = movie_df
    
    sampled_movies = genre_movies.sample(min(num, len(genre_movies)), replace=False)
    
    for _, row in sampled_movies.iterrows():
        movie_id = row['movieId']
        title = row['title']
        print(f"\n{title}")
        rating = input("How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: ")
        
        if rating.lower() == 'n':
            continue
        
        try:
            rating = float(rating)
            if 1 <= rating <= 5:
                rating_list.append({'userId': user_id, 'movieId': movie_id, 'rating': rating})
            else:
                print("Invalid rating. Please enter a number between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a number or 'n'.")
    
    return rating_list

In [18]:
# try out the new function here!
movie_df = pd.read_csv('./ml-latest-small/movies.csv')
user_ratings = movie_rater(movie_df, num=5, genre='Comedy')


Cars 3 (2017)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 2

Cloudy with a Chance of Meatballs 2 (2013)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 3

Mystery Train (1989)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: n

Miss Congeniality 2: Armed and Fabulous (2005)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 5

Ghost World (2001)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 3


If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [19]:
user_rating

NameError: name 'user_rating' is not defined

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [22]:
ratings_df = pd.read_csv('./ml-latest-small/movies.csv')
def add_new_ratings(original_ratings_df, new_ratings):
    new_ratings_df = pd.DataFrame(user_ratings)
    combined_df = pd.concat([original_ratings_df, new_ratings_df], ignore_index=True)
    return combined_df

combined_ratings_df = add_new_ratings(ratings_df, user_ratings)

In [23]:
# train a model using the new combined DataFrame
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split

def train_model(combined_ratings_df):
    reader = Reader(rating_scale=(1, 5))
    data = Dataset.load_from_df(combined_ratings_df[['userId', 'movieId', 'rating']], reader)
    trainset, testset = train_test_split(data, test_size=0.2)
    algo = SVD()
    algo.fit(trainset)
    return algo

algo = train_model(combined_ratings_df)

In [25]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
movies_df = pd.read_csv('./ml-latest-small/movies.csv')
def make_predictions(algo, movie_df, user_id):
    predictions = []
    for _, row in movie_df.iterrows():
        movie_id = row['movieId']
        prediction = algo.predict(user_id, movie_id)
        predictions.append((movie_id, prediction.est))
    return predictions

predictions = make_predictions(algo, movies_df, 1000)

In [26]:
# order the predictions from highest to lowest rated
def recommended_movies(predictions, movie_title_df, n):
    # Sort predictions by estimated rating
    ranked_movies = sorted(predictions, key=lambda x: x[1], reverse=True)[:n]
    for i, (movie_id, score) in enumerate(ranked_movies, start=1):
        title = movie_title_df[movie_title_df['movieId'] == movie_id]['title'].values[0]
        print(f"Recommendation #{i}: {title} (Score: {score:.2f})")

recommended_movies(predictions, movies_df, 5)

Recommendation #1: Toy Story (1995) (Score: 5.00)
Recommendation #2: Jumanji (1995) (Score: 5.00)
Recommendation #3: Grumpier Old Men (1995) (Score: 5.00)
Recommendation #4: Waiting to Exhale (1995) (Score: 5.00)
Recommendation #5: Father of the Bride Part II (1995) (Score: 5.00)


 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [27]:
# return the top n recommendations using the 
def run_recommender(ratings_df, movies_df, user_id=1000, num_ratings=5, n_recommendations=5, genre=None):
    # Step 1: Get user ratings
    print("Rate some movies:")
    user_ratings = movie_rater(movies_df, num_ratings, genre)

    # Step 2: Combine with original dataset
    combined_ratings_df = add_new_ratings(ratings_df, user_ratings)

    # Step 3: Train new model
    algo = train_model(combined_ratings_df)

    # Step 4: Make predictions
    predictions = make_predictions(algo, movies_df, user_id)

    # Step 5: Show recommendations
    print("\nTop Recommendations for You:")
    recommended_movies(predictions, movies_df, n_recommendations)

# Run the full recommender system
run_recommender(ratings_df, movies_df, num_ratings=5, n_recommendations=5, genre='Comedy')

Rate some movies:

No. 1 Ladies' Detective Agency, The (2008)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 2

Syrup (2013)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 1

Creature Comforts (1989)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 3

Le Maître d'école (1981)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 4

North (1994)
How do you rate this movie on a scale of 1-5, press 'n' if you have not seen: 5

Top Recommendations for You:
Recommendation #1: Toy Story (1995) (Score: 5.00)
Recommendation #2: Jumanji (1995) (Score: 5.00)
Recommendation #3: Grumpier Old Men (1995) (Score: 5.00)
Recommendation #4: Waiting to Exhale (1995) (Score: 5.00)
Recommendation #5: Father of the Bride Part II (1995) (Score: 5.00)


## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.